# Personalized Real-Estate Agent

In this notebook, we will build a personalized real-estate agent. 


## Import statements and configuration

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key is not None:
    print(f"Using API key from the `.env` file. - You're all set.")
else:
    print("OPENAI_API_KEY not found in environment variable. - Please set it up in the `.env` file.")

from openai import OpenAI
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key = openai_api_key,
)

MODEL_NAME = "gpt-3.5-turbo"
version = "v1"      # To make file names etc. unique
LISTINGS_FILE = f"listings_{version}.csv"

from langchain.llms import OpenAI

import pandas as pd
import numpy as np



Using API key from the `.env` file. - You're all set.


## Generating Real Estate Listings

First, we generate some fantasy listings using an LLM. The one example given is: 

In [7]:
listing_elements = ["Neighborhood", "Price", "Bedrooms", "Bathrooms", "House Size (in sqft)", "Description", "Neighborhood Description"]

listing_elements_text = ", ".join(listing_elements)

example_listing = """
"Green Oaks",800000,3,2,2000,"Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.","Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
"""

num_listings = 30
system_prompt = f"""
You are a real-estate listing generator. Your task is to create realistic and diverse real estate listings based on the provided example. Each listing should include the following fields: Neighborhood, Price, Bedrooms, Bathrooms, House Size (in sqft), Description, and Neighborhood Description. The listings should be varied in terms of price, size, and neighborhood features.
"""

user_prompt = f"""
Please generate {num_listings} real estate listings in the same format as the example below. The listings should be diverse and include various neighborhoods, prices, and features. Each listing should have a unique neighborhood description that highlights local amenities and attractions.
Listing features: 
{listing_elements_text}
Example Listing:
{example_listing}
"""
print(user_prompt)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response = client.responses.create(
    model=MODEL_NAME,
    messages=messages,
)
print(response.choices[0].message['content'])
print(response.output_text)




Please generate 30 real estate listings in the same format as the example below. The listings should be diverse and include various neighborhoods, prices, and features. Each listing should have a unique neighborhood description that highlights local amenities and attractions.
Listing features: 
Neighborhood, Price, Bedrooms, Bathrooms, House Size (in sqft), Description, Neighborhood Description
Example Listing:

"Green Oaks",800000,3,2,2000,"Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.","Green Oaks is a close-knit, e

NameError: name 'MODEL_NAME' is not defined

Next, we store the listings as a `.csv` file in order to retrieve them later.

If the listings are already generated, we can just read them from the csv file:

In [ ]:
df = pd.read_csv(LISTINGS_FILE)

## Storing Listings in a Vector Database

## Building the User Preference Interface

## Searching based on preferences

## Personalize Listing Descriptions